### Coding GPT from scratch:
(Only using pyTorch for neural network training and encoding)

This projects layouts the base structure and architecture for a language model, skipping the training and finetuning on large data (for obvious purposes). This is a character based LLM, rather than a whole word one, for simplicity purposes.

Its main purpose is to learn and consolidate my knowledge of the transformer architecture. I plan to follow it up, building the neural network from scratch (which I have already done in C), and then explore more advanced topics of LLMs, such as RAG, CAG, LPMs, etc.

In [8]:
import requests
import numpy as np

In [2]:
# Extracts the Tiny-Shakespeare and stores it in a variable:
url ="https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
r = requests.get(url)
if r.status_code == 200:
    text = r.text
    print("Dataset extracted. \n")
else:
    print(f"Failed to fetch. Status: {r.status_code}")

print(text[:200])


Dataset extracted. 

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [3]:
#Organizes the vocabulary (in this case unique characters, rather than words)
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(" \n Vocabulary size: "+ str(vocab_size))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
 
 Vocabulary size: 65


In [4]:
#Tokenizing the input text, encoding individual characters into vectors or integers

string_to_token = { ch: i for i, ch in enumerate(chars)} #creates dictionary with character as key and token as value
token_to_string = {i: ch for i, ch in enumerate(chars)} #creates dictionary with token as key and character as value

encode = lambda s : [string_to_token[c] for c in s] #given a string, give the corresponding series of tokens
decode = lambda l : ''.join([token_to_string[i] for i in l])#given a series of tokens, return the strings (concats into a single string)

print(encode("Hobbits"))
print(decode(encode("Hobbits")))
#Google uses Sentrencepeice for encoding, which encodes subwords.
#OpenAI uses tiktoken, which is a BPE tokenizer, with around 50k tokens.
#There is a trade-off between vocabulary size and token size. Thus subword is often used.

[20, 53, 40, 40, 47, 58, 57]
Hobbits


In [6]:
#Tokenizing the whole tinyshakespeare dataset, using pyTorch:
import torch
data = torch.tensor(encode(text), dtype = torch.long) # the data tensor is tikenized in its entirety
data.shape, data.dtype

(torch.Size([1115394]), torch.int64)

In [7]:
print(data[:200]) #tokenized the full dataset into a sequence of integers

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


In [ ]:
#splitting into training and validation sets, 90% for training
n = int(0.9*len(data)) #casts 0.9*len(data) to int
train_data = data[:n] #indexes 90% as training
val_data = data[n:] #the rest is validation, checking for overfitting

In [13]:
#feeding the text into the transformer by chunking the dataset and then sample it for training
block_size = 8 #AKA context window size
train_data[:block_size+1] #first context window of the dataset

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
#the training is done by guessing the next word, but for every single subsequence as well, in parallel
#the chunks of 9 characters actually only contain 8 examples, since the first word is the seed and must be present.
x = train_data[:block_size] #1st window, context
y = train_data[1:block_size+1] #2nd sliding window, it is the target
for t in range(block_size):
    context= x[:t+1]
    target = y[t]
    print(f"When input is {context}, the target is: {target}")

#The training is done from context 1 to block_size in parallel. It makes it more efficient to train. 
#And allows the transformer to percieve long and short term relationships between words, form 1 to block_size 

When input is tensor([18]), the target is: 47
When input is tensor([18, 47]), the target is: 56
When input is tensor([18, 47, 56]), the target is: 57
When input is tensor([18, 47, 56, 57]), the target is: 58
When input is tensor([18, 47, 56, 57, 58]), the target is: 1
When input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [19]:
#multiple blocks are computed in parallel into the transformer model, taking advantage of the GPUs.
#this batching is done for efficiency, and they are processed independently

torch.manual_seed(1337) #makes RNG stable for control and debugging purposes
batch_size = 4
block_size = 8

def get_batch(split):
    #generates the batch with inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) #generates batch_size random positions between 0 and the total indexable training set
    x = torch.stack([data[i:i+block_size] for i in ix])
    y= torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('Inputs:')
print(xb.shape)
print(xb)

print('Targets:')
print(yb.shape)
print(yb)
print("==============")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()}, the target is: {target}")

Inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
When input is [24], the target is: 43
When input is [24, 43], the target is: 58
When input is [24, 43, 58], the target is: 5
When input is [24, 43, 58, 5], the target is: 57
When input is [24, 43, 58, 5, 57], the target is: 1
When input is [24, 43, 58, 5, 57, 1], the target is: 46
When input is [24, 43, 58, 5, 57, 1, 46], the target is: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43], the target is: 39
When input is [44], the target is: 53
When input is [44, 53], the target is: 56
When input is [44, 53, 56], the target is: 1
When input is [44, 53, 56, 1], the target is: 58
When input is [44, 53, 56, 

In [20]:
#feeding it into a language model, the Bigram Language Model:

import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) #makes RNG stable for control and debugging purposes

class BigramLanguageModel(nn.Module):

    def __init__:
    

    def forward(self, idx, targets):

        